In [3]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np

import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian
import model_graddesc as model
import defmod.registrationdata as regdata

torch.set_default_tensor_type(torch.DoubleTensor)

In [4]:
data = regdata.organs()
data.build_modules()
data()
source = data.source
target = data.target

In [5]:
plt.figure()
plt.scatter(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), c='red')
plt.scatter(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), c='blue')
plt.axis('equal')

plt.figure()
plt.scatter(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), c='red')
plt.scatter(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), c='blue')

plt.axis('equal')
plt.show()

In [6]:
modules = dm.multishape.MultiShapeModule([data.modules[0], data.modules[1]])
modules.fill_controls_zero()
constr_Id = dm.constraints.Identity_Silent()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [7]:
# set parameters
reg_param = 10**-3 

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
    
EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id,source, target, 2, reg_param)

# do registration
X = [gd0_tensor, mom0_tensor]
X = model.gradientdescent(EnergyFunctional, X)

energy: constraints_________________
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [

cost: [ 0.00114817] attach: 14.242818831572475
c 0
 iter : 1  ,total energy: [ 14.243967]
energy: constraints_________________
tensor([[-1.3878e-17, -3.4694e-18],
        [ 1.7347e-17,  1.7347e-17],
        [ 2.0817e-17,  3.4694e-18],
        [-1.0408e-17, -1.7347e-18],
        [-1.3878e-17,  1.8648e-17],
        [-1.7347e-17,  3.7947e-18],
        [-3.1225e-17,  1.4745e-17],
        [-2.7756e-17,  1.3444e-17],
        [-3.1225e-17, -7.8063e-18],
        [-2.4286e-17,  5.2042e-18],
        [-2.0817e-17, -1.0408e-17],
        [ 0.0000e+00,  0.0000e+00],
        [-1.5613e-17, -3.4694e-18],
        [-1.2577e-17, -1.3878e-17],
        [ 3.4694e-18, -3.4694e-18],
        [ 1.3878e-17, -1.3878e-17],
        [ 1.0408e-17, -2.0817e-17],
        [ 1.7347e-17, -1.3878e-17],
        [ 0.0000e+00, -1.0408e-17],
        [ 3.4694e-18, -1.7347e-18],
        [ 1.7347e-17,  0.0000e+00],
        [ 3.4694e-18, -5.2042e-18],
        [ 3.4694e-18, -3.4694e-18],
        [ 1.3878e-17,  6.9389e-18],
        [

cost: [ 0.00641018] attach: 11.784952393127572
c 0
 iter : 3  ,total energy: [ 11.79136257]
energy: constraints_________________
tensor([[ 2.7756e-17, -2.7756e-17],
        [ 2.7756e-17,  0.0000e+00],
        [ 3.4694e-17,  4.8572e-17],
        [ 1.0408e-17,  1.3878e-17],
        [-2.0817e-17,  3.1225e-17],
        [-2.7756e-17,  4.5103e-17],
        [ 2.7756e-17,  1.8215e-17],
        [ 6.9389e-17,  8.6736e-18],
        [ 2.7756e-17,  5.2042e-17],
        [ 1.3878e-17,  3.4694e-18],
        [ 1.3878e-17,  2.7756e-17],
        [ 6.9389e-18,  6.2450e-17],
        [ 1.7347e-17, -1.3878e-17],
        [-1.2143e-17,  0.0000e+00],
        [ 1.3878e-17,  1.3878e-17],
        [ 1.3878e-17, -6.9389e-18],
        [ 1.3878e-17,  1.3878e-17],
        [ 0.0000e+00,  0.0000e+00],
        [ 1.3878e-17,  6.9389e-18],
        [ 0.0000e+00,  1.3878e-17],
        [ 6.9389e-18,  3.4694e-18],
        [ 0.0000e+00,  6.9389e-18],
        [-6.9389e-18, -1.0408e-17],
        [-2.7756e-17,  5.2042e-18],
       

cost: [ 0.01761966] attach: 9.758012130354034
c 0
 iter : 5  ,total energy: [ 9.77563179]
energy: constraints_________________
tensor([[-1.3878e-17,  0.0000e+00],
        [-5.5511e-17,  1.9429e-16],
        [ 4.1633e-17,  0.0000e+00],
        [ 2.7756e-17, -2.0817e-17],
        [-4.1633e-17,  7.6328e-17],
        [ 2.7756e-17,  3.7730e-17],
        [-5.5511e-17,  4.0766e-17],
        [ 8.3267e-17,  4.6838e-17],
        [ 2.7756e-17,  1.0061e-16],
        [-2.7756e-17,  3.4694e-17],
        [ 0.0000e+00,  2.7756e-17],
        [-2.7756e-17,  5.5511e-17],
        [ 1.3878e-17,  8.3267e-17],
        [-4.5103e-17,  1.3878e-16],
        [-2.7756e-17,  4.1633e-17],
        [-4.1633e-17,  2.7756e-17],
        [ 1.3878e-17,  6.9389e-17],
        [-2.7756e-17,  4.1633e-17],
        [-2.7756e-17,  4.1633e-17],
        [ 2.7756e-17,  2.7756e-17],
        [-5.5511e-17,  0.0000e+00],
        [ 0.0000e+00,  4.1633e-17],
        [-1.3878e-17,  3.4694e-18],
        [-2.7756e-17,  3.4694e-17],
        [

cost: [ 0.02360817] attach: 9.302947861516259
c 0
 iter : 7  ,total energy: [ 9.32655604]
energy: constraints_________________
tensor([[-2.7756e-17,  2.7756e-17],
        [ 5.5511e-17,  5.5511e-17],
        [ 0.0000e+00,  1.3878e-17],
        [ 0.0000e+00, -2.0123e-16],
        [ 0.0000e+00,  3.4694e-18],
        [-5.5511e-17,  5.0307e-17],
        [-5.5511e-17, -3.2960e-17],
        [-2.7756e-17, -7.0256e-17],
        [ 5.5511e-17, -7.6328e-17],
        [-2.7756e-17,  2.7756e-17],
        [-2.7756e-17, -6.9389e-17],
        [ 0.0000e+00,  2.7756e-17],
        [ 1.3184e-16,  0.0000e+00],
        [-1.7347e-17, -8.3267e-17],
        [ 5.5511e-17, -5.5511e-17],
        [ 8.3267e-17, -5.5511e-17],
        [ 2.7756e-17,  0.0000e+00],
        [ 8.3267e-17, -2.7756e-17],
        [-5.5511e-17,  5.5511e-17],
        [ 2.7756e-17, -5.5511e-17],
        [ 2.7756e-17,  1.3878e-17],
        [ 2.7756e-17, -1.3878e-17],
        [-2.7756e-17, -1.3878e-17],
        [-5.5511e-17,  5.5511e-17],
        [

...

cost: [ 0.35376955] attach: 2.32296524049809
Condition 0: tensor([1], dtype=torch.uint8)
Condition 1, 2, 3: tensor([1], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor([0], dtype=torch.uint8)
Contition 4: tensor(0, dtype=torch.uint8)
Condition 5: True
c 1
 iter : 191  ,total energy: [ 2.67673479]
energy: constraints_________________
tensor([[ 4.4409e-16,  2.2204e-16],
        [ 8.8818e-16, -1.1102e-15],
        [-2.2204e-16,  0.0000e+00],
        [-9.9920e-16, -1.1102e-15],
        [-6.6613e-16,  2.2204e-16],
        [ 1.3323e-15, -9.7838e-16],
        [-4.4409e-16, -4.4409e-16],
        [-4.4409e-16, -5.4123e-16],
        [-8.8818e-16, -1.0547e-15],
        [-4.4409e-16, -1.2212e-15],
        [ 0.0000e+00, -1.9984e-15],
        [-2.2204e-16,  4.4409e-16],
        [ 7.7716e-16, -2.2204e-15],
        [ 1.1102e-16, -8.8818e-16],
        [ 7.7716e-16, -8.8818e-16],
        [ 1.3323e-15,  6.6613e-16],
        [ 4.4409e-16, -1.1102e-15],
        [ 4.4409e-16, -1.1102e-15],
        [ 

cost: [ 0.35392902] attach: 2.3122421859441302
Condition 0: tensor([0], dtype=torch.uint8)
Condition 1, 2, 3: tensor([0], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor([0], dtype=torch.uint8)
Contition 4: tensor(0, dtype=torch.uint8)
Condition 5: True
c 0
 iter : 193  ,total energy: [ 2.6661712]
energy: constraints_________________
tensor([[ 8.8818e-16, -6.6613e-16],
        [ 2.2204e-16,  4.4409e-16],
        [ 7.7716e-16,  4.4409e-16],
        [-2.2204e-16,  4.4409e-16],
        [ 1.1102e-15, -4.1633e-16],
        [ 0.0000e+00, -1.0894e-15],
        [ 8.8818e-16, -1.6792e-15],
        [ 1.3323e-15,  8.0491e-16],
        [ 2.2204e-15, -3.8858e-16],
        [ 8.8818e-16, -7.7716e-16],
        [ 1.3323e-15,  2.2204e-16],
        [ 0.0000e+00,  2.2204e-16],
        [ 1.1102e-15,  4.4409e-16],
        [ 4.9960e-16, -4.4409e-16],
        [-3.3307e-16, -8.8818e-16],
        [-4.4409e-16, -4.4409e-16],
        [ 4.4409e-16,  4.4409e-16],
        [ 4.4409e-16,  0.0000e+00],
        [

cost: [ 0.3566859] attach: 2.3057149838571185
Condition 0: tensor([0], dtype=torch.uint8)
Condition 1, 2, 3: tensor([0], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor([0], dtype=torch.uint8)
Contition 4: tensor(0, dtype=torch.uint8)
Condition 5: True
c 0
 iter : 195  ,total energy: [ 2.66240088]
energy: constraints_________________
tensor([[ 4.4409e-16, -8.8818e-16],
        [ 8.8818e-16,  6.6613e-16],
        [ 2.1094e-15, -2.2204e-16],
        [ 7.7716e-16, -1.6653e-15],
        [ 8.8818e-16, -3.0531e-16],
        [ 1.9984e-15, -1.3392e-15],
        [-4.4409e-16, -6.9389e-16],
        [-4.4409e-16, -7.7716e-16],
        [ 0.0000e+00, -3.8858e-16],
        [ 8.8818e-16, -4.4409e-16],
        [ 4.4409e-16, -6.6613e-16],
        [-2.2204e-16,  0.0000e+00],
        [ 3.3307e-16, -8.8818e-16],
        [ 4.4409e-16, -8.8818e-16],
        [ 1.1102e-16, -1.3323e-15],
        [ 2.2204e-16, -2.2204e-16],
        [ 4.4409e-16, -4.4409e-16],
        [ 2.2204e-16,  2.2204e-16],
        [

cost: [ 0.3606846] attach: 2.318765433077673
Condition 0: tensor([1], dtype=torch.uint8)
Condition 1, 2, 3: tensor([1], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor([0], dtype=torch.uint8)
Contition 4: tensor(0, dtype=torch.uint8)
Condition 5: True
c 1
 iter : 197  ,total energy: [ 2.67945003]
energy: constraints_________________
tensor([[-1.7764e-15,  1.1102e-15],
        [ 0.0000e+00, -8.8818e-16],
        [ 1.6653e-16,  1.3323e-15],
        [ 1.4433e-15, -1.4433e-15],
        [ 1.7764e-15,  5.8287e-16],
        [ 2.4425e-15, -8.3961e-16],
        [ 1.3323e-15, -3.6082e-16],
        [ 4.4409e-16, -1.3357e-15],
        [ 0.0000e+00, -9.4369e-16],
        [-8.8818e-16, -8.8818e-16],
        [ 8.8818e-16,  6.6613e-16],
        [ 0.0000e+00,  4.4409e-16],
        [ 4.4409e-16, -8.8818e-16],
        [ 4.9960e-16, -8.8818e-16],
        [ 8.8818e-16,  4.4409e-16],
        [ 2.2204e-16, -4.4409e-16],
        [ 2.2204e-16, -8.8818e-16],
        [ 2.2204e-16,  2.2204e-16],
        [ 

cost: [ 0.3608404] attach: 2.297568464304291
Condition 0: tensor([0], dtype=torch.uint8)
Condition 1, 2, 3: tensor([0], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor([0], dtype=torch.uint8)
Contition 4: tensor(0, dtype=torch.uint8)
Condition 5: True
c 0
 iter : 199  ,total energy: [ 2.65840887]
energy: constraints_________________
tensor([[ 0.0000e+00, -2.2204e-16],
        [-2.2204e-16, -2.2204e-16],
        [ 8.3267e-16,  4.4409e-16],
        [-4.4409e-16, -6.6613e-16],
        [ 0.0000e+00,  3.6082e-16],
        [-2.2204e-15,  1.1172e-15],
        [-4.4409e-16, -2.0817e-16],
        [ 0.0000e+00, -1.5023e-15],
        [-4.4409e-16, -1.6653e-16],
        [ 4.4409e-16, -8.8818e-16],
        [-8.8818e-16, -6.6613e-16],
        [-8.8818e-16, -6.6613e-16],
        [-7.7716e-16, -8.8818e-16],
        [ 2.7756e-16,  8.8818e-16],
        [-1.1102e-16, -8.8818e-16],
        [ 4.4409e-16,  6.6613e-16],
        [-8.8818e-16,  0.0000e+00],
        [-4.4409e-16,  2.2204e-16],
        [ 

In [9]:
#%matplotlib qt5
H.module.manifold.fill_gd(X[0])
H.module.manifold.fill_cotan(X[1])

states, controls = dm.shooting.shoot_euler(H, it=10)

import matplotlib.cm as cm
colors = cm.rainbow(np.linspace(0, 1, len(states)))

plt.figure()
for s,c in zip(states, colors):
    plt.scatter(s[0][0].gd.view(-1,2)[:, 0].detach().numpy(), s[0][0].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[1][0].gd.view(-1,2)[:, 0].detach().numpy(), s[1][0].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[2][0].gd.view(-1,2)[:, 0].detach().numpy(), s[2][0].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
    plt.scatter(s[2][1].gd.view(-1,2)[:, 0].detach().numpy(), s[2][1].gd.view(-1,2)[:, 1].detach().numpy(), c=c)
plt.plot(target[0][:, 0].detach().numpy(), target[0][:, 1].detach().numpy(), 'xk')
plt.plot(target[1][:, 0].detach().numpy(), target[1][:, 1].detach().numpy(), 'xk')

plt.plot(source[0][:, 0].detach().numpy(), source[0][:, 1].detach().numpy(), '.k')
plt.plot(source[1][:, 0].detach().numpy(), source[1][:, 1].detach().numpy(), '.k')

    
plt.axis('equal')
plt.show()

In [ ]:
# Visualization of grid deformation for each submodule
nx, ny = 30, 30
sx, sy = 30, 30
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.DoubleTensor)/nx - 0.5)
y = sy*(y.type(torch.DoubleTensor)/ny - 0.5)

gridpoints = dm.usefulfunctions.grid2vec(x, y).type(torch.DoubleTensor)


phi = mm.largeDeformation(modules, states, controls, gridpoints)
x1, y1 = dm.usefulfunctions.vec2grid(phi[0], nx, ny)
x2, y2 = dm.usefulfunctions.vec2grid(phi[1], nx, ny)
x3, y3 = dm.usefulfunctions.vec2grid(phi[2], nx, ny)



mm.plot_grid( x1.detach().numpy(), y1.detach().numpy(), color = 'blue')
mm.plot_grid(x2.detach().numpy(), y2.detach().numpy(), color = 'blue')
mm.plot_grid( x3.detach().numpy(), y3.detach().numpy(), color = 'blue')
plt.show()


In [11]:
modules.module_list[0].module_list

In [ ]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
                
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), rais\gride_exception=True)

gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)


test_gradcheck_Hamiltonian()

In [ ]:
def test_gradcheck_EnergyFunctional():
    def energy(gd, mom):

        return model.EnergyFunctional(H.module, H, constr_Id,source, target, 2, reg_param).energy_tensor(gd, mom)
    return torch.autograd.gradcheck(energy, (gd, mom), raise_exception=True)

gd = gd0_tensor
mom = mom0_tensor


test_gradcheck_EnergyFunctional()

In [ ]:
def test_gradcheck_shooting():
    def shooting(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        
        intermediate_states, intermediate_controls = dm.shooting.shoot_euler(H, it=2)
        return H.module.manifold.unroll_cotan()
    return torch.autograd.gradcheck(shooting, (gd, mom), raise_exception=True,atol=1e-2)

test_gradcheck_shooting()

In [ ]:
def test_gradcheck_attachement():
    def attach(gd, mom):
        reg_param = 10**-3 

        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        energyfunc = model.EnergyFunctional(H.module, H, constr_Id,source, target, 2, reg_param)

        return energyfunc.attach() 
    return torch.autograd.gradcheck(attach, (gd, mom), raise_exception=True)

gd = torch.cat(H.module.manifold.unroll_gd(),0).requires_grad_().view(-1)
mom = torch.cat(H.module.manifold.unroll_cotan(),0).requires_grad_().view(-1)

test_gradcheck_attachement()

In [ ]:
def test_gradcheck_EnergyFunctional():
    def energy(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.module.compute_geodesic_variables(constr_Id)
        return H.module.controls[0]
    return torch.autograd.gradcheck(energy, (gd, mom), raise_exception=True)

gd = torch.cat(H.module.manifold.unroll_gd(),0).requires_grad_().view(-1)
mom = torch.cat(H.module.manifold.unroll_cotan(),0).requires_grad_().view(-1)

test_gradcheck_EnergyFunctional()

In [ ]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        return mod.autoaction()
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = data.modules[0].copy()
gd = torch.cat(mod.manifold.gd.copy())
mom = torch.cat(mod.manifold.cotan.copy())

gd.requires_grad
mom.requires_grad

test_gradcheck_autoaction()